In [1]:
import sys
import pickle
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import os
from torch_geometric.data import Dataset, download_url, Data
import pandas as pd
import shutil
import torch
from scipy.spatial import distance

from torch_geometric import nn as gnn
import torch.nn.functional as F
from torchvision.transforms.functional import crop
from torch import nn

/home/paulraae/.conda/envs/ms_thesis_Env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
with open('/home/paulraae/MS_Thesis_Data/tester_eye_data/test_202_with_tiles_processed.pk1', 'rb') as handle:
    data = pickle.load(handle)

In [32]:
class One_graph_rest_free(Dataset): #For this ds I have ald done all the required pre prcessing 
    def __init__(self, root, data_dict = None):
        self.data_dict = data_dict
        super().__init__(root, None, None, None)
        
        
    @property
    def raw_file_names(self):
        return "not_implementated.pt"

    @property
    def processed_file_names(self):
        return "not_implemented.pt"
    
    def process(self):
            # Process pre made data dictionary

            centroids = torch.Tensor(self.data_dict["tile_centroids"].reshape((12,56,56,3)))
            tiles = torch.Tensor(self.data_dict["tiles"].reshape((3232,56,56,3)))
            tile_clusters = torch.Tensor(self.data_dict["tile_cluster"])
            centroid_y = torch.Tensor(torch.Tensor([i for i in range(self.data_dict["tile_centroids"].shape[0])]))
            tile_knn_edges = torch.Tensor(self.data_dict["tile_knn_edge_index"])
            
            x_features = torch.cat((tiles, centroids))
            y = torch.cat((tile_clusters, centroid_y)).to(torch.int)
            
            edge_map = []


            i=-1
            for cluster in y[:tiles.shape[0]]:
                i+=1
                edge_map.append([i, cluster+tiles.shape[0]])
            

            i+=1
            for centroid1 in y[i:]:
                for centroid2 in y[i+1:]:
                    edge_map.append([centroid1+tiles.shape[0], centroid2+tiles.shape[0]])
                i+=1
            
            edge_map = torch.cat((torch.Tensor(edge_map), tile_knn_edges), axis=0)
            
            edge_map = edge_map.to(int)
            
            edge_map_aux = [[],[]]
            for x in edge_map:
                edge_map_aux[0].append(x[0])
                edge_map_aux[1].append(x[1]) 
                    
            edge_map_aux = np.array(edge_map_aux)
            
            edge_attrs = []
            for _ in edge_map_aux[0]:
                edge_attrs.append(1)
                
            edge_attrs = torch.Tensor(edge_attrs)
            edge_map_aux = torch.Tensor(edge_map_aux).to(int)
            
            data = Data(x_features, edge_map_aux, edge_attrs, y)

            torch.save(data, os.path.join(self.processed_dir, f'data_0.pt'))

    def len(self):
        return self.data_dict["features"].shape[0]

    def get(self, idx):
        data = torch.load(os.path.join(self.processed_dir, f'data_{idx}.pt'))
        return data

In [33]:
root_path = "/home/paulraae/MS_Thesis/ViG_based_link_pred_implementation/test_graph_ds"
dataset = One_graph_rest_free(root_path, data_dict=data)

Processing...


Done!


In [55]:
dataset[0]

Data(x=[3244, 56, 56, 3], edge_index=[2, 16541], edge_attr=[16541], y=[3244])

In [52]:
import sys
import os

sys.path.append('/home/paulraae/')
from Vision_GNN.vig import *

from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from timm.models.helpers import load_pretrained
from timm.models.layers import DropPath, to_2tuple, trunc_normal_
from timm.models.registry import register_model

In [ ]:
patch_size = 56

def _cfg(url='', **kwargs):
    return {
        'url': url,
        'num_classes': 1000, 'input_size': (3, 224, 224), 'pool_size': None,
        'crop_pct': .9, 'interpolation': 'bicubic',
        'mean': IMAGENET_DEFAULT_MEAN, 'std': IMAGENET_DEFAULT_STD,
        'first_conv': 'patch_embed.proj', 'classifier': 'head',
        **kwargs
    }


default_cfgs = {
    'gnn_patch16_224': _cfg(
        crop_pct=0.9, input_size=(3, 224, 224),
        mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5),
    ),
}

@register_model
def vig_ti_224_gelu(pretrained=False, **kwargs):
    class OptInit:
        def __init__(self, num_classes=1000, drop_path_rate=0.0, drop_rate=0.0, num_knn=9, **kwargs):
            self.k = num_knn # neighbor num (default:9)
            self.conv = 'mr' # graph conv layer {edge, mr}
            self.act = 'gelu' # activation layer {relu, prelu, leakyrelu, gelu, hswish}
            self.norm = 'batch' # batch or instance normalization {batch, instance}
            self.bias = True # bias of conv layer True or False
            self.n_blocks = 12 # number of basic blocks in the backbone
            self.n_filters = 192 # number of channels of deep features
            self.n_classes = num_classes # Dimension of out_channels
            self.dropout = drop_rate # dropout rate
            self.use_dilation = True # use dilated knn or not
            self.epsilon = 0.2 # stochastic epsilon for gcn
            self.use_stochastic = False # stochastic for gcn, True or False
            self.drop_path = drop_path_rate

    opt = OptInit(**kwargs)
    model = DeepGCN(opt)
    model.default_cfg = default_cfgs['gnn_patch16_224']
    return model

In [13]:
model = vig_block(4096,4096,4096,4)

In [15]:
test_out = model.forward(dataset[0].x["graph"], dataset[0].edge_index)


In [16]:
test_out.shape

torch.Size([132, 4096])